## 01. 이미지 암/복호화

In [2]:
import cv2
import numpy as np
from Cryptodome import Random
from Cryptodome.Cipher import AES
from Cryptodome.Util import Padding
import matplotlib.pyplot as plt
import os,glob


#### 01-1. 암호화/복호화 함수

In [ ]:
class Cipher:
    def __init__(self,key,mode=AES.MODE_GCM):
        self.key = Padding.pad(str(key).encode(),16)
        #self.key = key
        self.mode = mode
        print(self.key)
        
    def encrypt(self,crop_image):
        _, png_image = cv2.imencode('.png',crop_image)
        byte_image = png_image.tobytes()
        
        iv = Random.new().read(AES.block_size)
        cipher = AES.new(self.key, self.mode, iv)
        
        return iv+cipher.encrypt(byte_image)
    
    def decrypt(self, enc_data):
        iv = enc_data[:16]
        cipher = AES.new(self.key,self.mode,iv)
        dec_result = cipher.decrypt(enc_data[16:])
        box_array = np.frombuffer(dec_result, dtype='uint8')
        box_image = cv2.imdecode(box_array, cv2.IMREAD_COLOR)
        
        return box_image
        

#### 01-2. 키설정

In [3]:
import string

In [4]:
def key_generator(size = 16, chars = string.ascii_lowercase):
    return ''.join(random.choice(chars) for _ in range(size))

In [ ]:
key = key_generator(16)

#### 01-3. 이미지로딩

In [ ]:
path = 'C:/Users/User/Desktop/img_dataset'
filename = path+'001c2a14-c7138401.jpg'
img = cv2.imread(filename)
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()

#### 01-4. 적용

In [ ]:
Cipher(bytes(key,encoding='utf8')).encrypt(img)

In [ ]:
Cipher(bytes(key,encoding='utf8')).decrypt(Cipher(bytes(key,encoding='utf8')).encrypt(img))

## 02. 텍스트암호화
- pip install pycryptodomex

In [ ]:
import base64
from Crypto import Random
from Crypto.Cipher import AES

#### 02-1. 블럭사이즈에 대한 패딩로직
- 스트링 문자열을 encrypt에 인자로 전달시 입력받은 데이터의 길이가 BLOCK_SIZE의 배수가 아닐때 패딩개념이 필요하다. AES에서는 BLOCK_SIZE가 128bit 즉 16바이트로 고정되는데, 아래 코드를 통해 자동 패딩처리한다. 한글에 대한 처리를 위해 pad시에 len(s.encode('utf-8')) 처리가 반드시 필요하다. 영문과 기호는 문자당 1바이트지만 한글은 문자당 2바이트이기 때문이다. len()함수를 활용해 길이를 통해 바이트 계산을 하는 방식이므로 ‘utf-8’ 변환하지 않을 경우 오류가 발생하게 된다.

In [ ]:
BS = 16
pad = lambda s: s + (BS - len(s.encode('utf-8')) % BS) * chr(BS - len(s.encode('utf-8')) % BS)
unpad = lambda s : s[:-ord(s[len(s)-1:])]

#### 02-2. 암호화알고리즘 클래스 생성
- encrypt 함수의 내부를 보면 입력받은 raw데이터를 raw.encode(‘utf-8’) 16바이트 바이너리로 변환 후, 다시 base64.b64encode() 인코딩 한다. 즉 문자열을 기준으로 해서 byte화시키는 문자인코딩과는 다르게 bytes를 기준으로 문자열화 시키는 인코딩 방식 이 필요하다.

In [ ]:
class AESCipher:
    def __init__( self, key ):
        self.key = key

    def encrypt( self, raw ):
        raw = pad(raw)
        iv = Random.new().read( AES.block_size )
        cipher = AES.new( self.key, AES.MODE_CBC, iv )
        return base64.b64encode( iv + cipher.encrypt( raw.encode('utf-8') ) )

    def decrypt( self, enc ):
        enc = base64.b64decode(enc)
        iv = enc[:16]
        cipher = AES.new(self.key, AES.MODE_CBC, iv )
        return unpad(cipher.decrypt( enc[16:] ))

#### 02-3. 키설정
- AES256을 구현하기 위해 32바이트의 키를 생성한다. ( 2 ^32^ = 256) KEY값은 아래와 같이 리스트 베열로 바로 지정하거나, time함수와 hashlib를 통해 임의로 생성해볼 수 있다.

###### 02-3-1. 리스트에 세팅하는 방식

In [12]:
key = [0x10, 0x01, 0x15, 0x1B, 0xA1, 0x11, 0x57, 0x72, 0x6C, 0x21, 0x56, 0x57, 0x62, 0x16, 0x05, 0x3D,
        0xFF, 0xFE, 0x11, 0x1B, 0x21, 0x31, 0x57, 0x72, 0x6B, 0x21, 0xA6, 0xA7, 0x6E, 0xE6, 0xE5, 0x3F]

###### 02-3-2. 함수 방식(1)

In [13]:
def key_generator(size = 16, chars = string.ascii_lowercase):
    return ''.join(random.choice(chars) for _ in range(size))

In [ ]:
key = key_generator(16)

###### 02-3-2. 함수 방식(2)

In [ ]:
import base64
import hashlib

def make_pass():
    timekey = int(time.time())
    return str(timekey)

password = make_pass().encode('utf-8')

#### 02-4. 암호화/복호화

In [15]:
data = "Iran has seized a foreign oil tanker in the Persian Gulf that was smuggling fuel to some Arab states, according to a state television report on Sunday. The report said that the tanker had been detained and the ship's foreign crew held by the country's elite Islamic Revolutionary Guards Corps."

In [ ]:
encrypted_data = AESCipher(bytes(key)).encrypt(data)  
encrypted_data

In [ ]:
decrypted_data = AESCipher(bytes(key)).decrypt(encrypted_data)
decrypted_data.decode('utf-8')